In [18]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import BertTokenizer
from loralib.layers import Linear
from loralib.utils import mark_only_lora_as_trainable

upload dataset

In [19]:
dataset = load_dataset("ag_news")
print(dataset["train"][0])

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}


BERT pre-training

In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # BERT分词器分词

def encode_with_bert(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding='max_length',
        max_length=128,
    )
    return {
        "input_ids": tokens["input_ids"],
        "attention_mask": tokens["attention_mask"],
        "label": example["label"]
    }

encoded_dataset = dataset.map(encode_with_bert, batched=True) # Hugging Face Datasets库的map函数批量处理数据

encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"]) # 设置数据格式为PyTorch张量，并指定需要的列


Prepare DataLoader

In [21]:
train_loader = DataLoader(encoded_dataset["train"], batch_size=32, shuffle=True)
test_loader = DataLoader(encoded_dataset["test"], batch_size=32)

Define RNN model with LoRA classification head

In [22]:
class LoRALSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.i2h = Linear(input_size, 4 * hidden_size, r=8)
        self.h2h = Linear(hidden_size, 4 * hidden_size, r=8)

    def forward(self, x, hidden):
        h, c = hidden
        gates = self.i2h(x) + self.h2h(h)
        i, f, g, o = gates.chunk(4, 1)
        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        g = torch.tanh(g)
        o = torch.sigmoid(o)
        c_next = f * c + i * g
        h_next = o * torch.tanh(c_next)
        return h_next, c_next
    
class TextLSTMWithLoRA(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm_cell = LoRALSTMCell(embed_dim, hidden_dim)
        self.hidden_dim = hidden_dim
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, input_ids, attention_mask=None):
        embedded = self.embedding(input_ids)
        batch_size, seq_len, _ = embedded.size()
        h_t = torch.zeros(batch_size, self.hidden_dim, device=embedded.device)
        c_t = torch.zeros(batch_size, self.hidden_dim, device=embedded.device)

        for t in range(seq_len):
            h_t, c_t = self.lstm_cell(embedded[:, t, :], (h_t, c_t))

        out = self.fc(h_t)
        return out


In [23]:
# Initialize model
model_LSTM = TextLSTMWithLoRA(vocab_size=tokenizer.vocab_size, embed_dim=128, hidden_dim=128, num_classes=4)
# Load pretrained weights from IMDB except fc
state_dict = torch.load("lstm_imdb.pth")
filtered_dict = {k: v for k, v in state_dict.items() if not k.startswith("fc")}
model_LSTM.load_state_dict(filtered_dict, strict=False)

# freeze all except LoRA
mark_only_lora_as_trainable(model_LSTM)

# Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_LSTM.to(device)

# Define optimizer and loss
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model_LSTM.parameters()), lr=1e-3)
criterion = nn.CrossEntropyLoss()


Train model

In [24]:
for epoch in range(20):
    model_LSTM.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        outputs = model_LSTM(input_ids)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1, Loss: 1.3067
Epoch 2, Loss: 1.1356
Epoch 3, Loss: 1.0458
Epoch 4, Loss: 0.9809
Epoch 5, Loss: 0.9267
Epoch 6, Loss: 0.8857
Epoch 7, Loss: 0.8478
Epoch 8, Loss: 0.8166
Epoch 9, Loss: 0.7949
Epoch 10, Loss: 0.7734
Epoch 11, Loss: 0.7508
Epoch 12, Loss: 0.7321
Epoch 13, Loss: 0.7192
Epoch 14, Loss: 0.7008
Epoch 15, Loss: 0.6804
Epoch 16, Loss: 0.6659
Epoch 17, Loss: 0.6538
Epoch 18, Loss: 0.6417
Epoch 19, Loss: 0.6341
Epoch 20, Loss: 0.6242


Evaluation

In [25]:
model_LSTM.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        outputs = model_LSTM(input_ids)
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {correct / total:.2%}")



Test Accuracy: 76.51%
